Importando as bibliotecas necessárias para a criação das análises exploratórias e do modelo de classificação

In [ ]:
#biblioteca pandas para manipulação do banco de dados
import pandas as pd
#numpy para realização de cálculos
import numpy as np
#biblioteca matplotlib para criação dos gráficos
import matplotlib.pyplot as plt
#biblioteca seaborn para criação dos gráficos
import seaborn as sns
#criar o ordinal encoder
from sklearn.preprocessing import OrdinalEncoder
#padroniza as variáveis
from sklearn.preprocessing import StandardScaler
#regressão logística
from sklearn.linear_model import LogisticRegression
#Validação do modelo por grid e cross validation
from sklearn.model_selection import GridSearchCV, StratifiedKFold
#modelo random forest
from sklearn.ensemble import RandomForestClassifier
#biblioteca sklearn.metrics para definição das métricas de avaliação do modelo
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score,\
classification_report, roc_auc_score, roc_curve, recall_score

%matplotlib inline

Importando os dados

In [ ]:
#armazena o caminho dos dados em uma variável
caminho = 'base_churn.xlsx'
#lendo o arquivo excel
dados_brutos = pd.read_excel(caminho)
#um head() dos dados contendo as 5 primeiras linhas
dados_brutos.head()

Análise exploratória dos dados

In [ ]:
#lista de colunas onde queremos fazer o replace
colunas_para_alterar = ['INADIMPLENTE', 'SEXO']

# Substituir nas colunas especificadas
dados_brutos[colunas_para_alterar] = dados_brutos[colunas_para_alterar].replace({
    'SIM': 'INADIMPLENTE',
    'NÃO': 'NAO_INADIMPLENTE',
    'M': 'MASCULINO',
    'F': 'FEMININO'
    })

In [ ]:
#avalia os tipos de variáveis e se alguma precisa ser modificada
dados_brutos.info()

In [ ]:
#quantidade de linhas e colunas do banco de dados
dados_brutos.shape

In [ ]:
#renomeando a coluna IDADENAADESÃO
dados = dados_brutos.rename(columns={'IDADENAADESÃO': 'IDADE_NA_ADESAO'})
dados.head(1)

In [ ]:
#tabela de frequência
def tabela_de_frequencias(dados):
  resultados = []

  for i in dados.select_dtypes(include=['object']).columns:
    value_counts = dados[i].value_counts().sort_index().reset_index()
    value_counts.columns = ['Valor', 'Frequencia']
    value_counts['Variavel'] = i  # Adiciona uma coluna com o nome da coluna
    resultados.append(value_counts)

  dados_resultados = pd.concat(resultados, ignore_index=True)
  return dados_resultados

In [ ]:
#tabela de frequência
dados_resultados = tabela_de_frequencias(dados)
print("\033[92m Tabela de Frequência\n")
dados_resultados

In [ ]:
# calculando média de BMI agrupada por Stroke
media_agrupada_bmi = dados.groupby('CANCELADO')['VALOR_MENSALIDADE'].mean().round(2)
media_agrupada_bmi_df = media_agrupada_bmi.reset_index()

# convertendo a variavel resposta em objeto
media_agrupada_bmi_df['CANCELADO'] = media_agrupada_bmi_df['CANCELADO'].astype(object)
media_agrupada_bmi_df.columns = ['CANCELADO', 'Média VALOR_MENSALIDADE']
display(media_agrupada_bmi_df)

In [ ]:
#variáveis categoricas
dados['FAIXA_RENDA'].unique()

In [ ]:
#categorias salvas
categorias =  ['Baixa renda', 'Média renda', 'Alta renda']

In [ ]:
#ordena as categorias
enc = OrdinalEncoder(categories=[categorias])

In [ ]:
#aplica a ordenação na variável desejada
dados['FAIXA_RENDA'] = enc.fit_transform(dados[['FAIXA_RENDA']])
dados.head()

In [ ]:
#cria e recodifica uma nova variável resposta
#CANCELADO
dados.loc[dados['CANCELADO'] != "NÃO" , 'CANCELADO_USAR'] = 1
dados.loc[dados['CANCELADO'] == "NÃO", 'CANCELADO_USAR'] = 0
#FAIXO DE RENDA
dados.loc[dados['FAIXA_RENDA'] == 0 , 'FAIXA_RENDA_USAR'] = "Baixa renda"
dados.loc[dados['FAIXA_RENDA'] == 1, 'FAIXA_RENDA_USAR'] = "Média renda"
dados.loc[dados['FAIXA_RENDA'] == 2, 'FAIXA_RENDA_USAR'] = "Alta renda"
dados.head()

In [ ]:
dados.info()

In [ ]:
#estatísticas univariadas (ID_CLIENTE não é variável conínua)
dados.describe()

*Observações: existem valores negativos para a idade e tempo de plano, possivelmente foi um erro de digitação.*

---



*Médias*

*Tempo de plano em meses: 258*


*Consultas em 12 meses: 12*


*Internações em 12 meses: 1,3*


*Valor mensal: 916 reais*

Verificando valores faltantes


In [ ]:
#quantidade de valores negativos
negativos = dados[dados['IDADE_NA_ADESAO'] < 0]
negativos['IDADE_NA_ADESAO'].value_counts()

In [ ]:
#convertendo valores negativos em positivos
dados['IDADE_NA_ADESAO'] = dados['IDADE_NA_ADESAO'].abs()
dados['TEMPO_DE_PLANO_MESES'] = dados['TEMPO_DE_PLANO_MESES'].abs()

In [ ]:
#investigando se ocorreu tudo bem para a variável idade
negativos1 = dados[dados['IDADE_NA_ADESAO'] < 0]
negativos1['IDADE_NA_ADESAO'].value_counts()

In [ ]:
#avalia se ocorreu tudo bem
negativos2 = dados[dados['TEMPO_DE_PLANO_MESES'] < 0]
negativos2['TEMPO_DE_PLANO_MESES'].value_counts()


In [ ]:
#estatísticas univariadas (ID_CLIENTE e CANCELADO_usar não é variável conínua) para confirmar a conversão em valores negativos em absolutos
dados.describe()

In [ ]:
#nomes das colunas do tipo "objeto"
dados.select_dtypes(include="object").columns

In [ ]:
#quantidade de valores faltantes em cada variável
print("\033[92m \n--------------Valores Absolutos--------------------\n", "\033[97m \n", dados.isna().sum())
print("\033[92m \n--------------Valores Percentuais--------------------\n", "\033[97m \n", dados.isna().sum()/dados.shape[0]*100)

*Observações: existem valores faltantes nas variáveis SEXO, UF e VALOR MENSALIDADE, porém temos um percentual maior em valor da mensalidade então iremos inputar valores utilizando o KNN e testar. Se removermos as linhas com valores faltantes será removido mais de 36% do tamanho do banco de dados, perdendo assim, muita informação.*

Tratamento de valores faltantes

In [ ]:
#biblioteca KNN para imputar valores com base nos valores vizinhos
from sklearn.impute import KNNImputer
#função em uma variável
impute = KNNImputer(n_neighbors=5)

In [ ]:
#imputa os valores
dados['VALOR_MENSALIDADE'] = impute.fit_transform(dados[['VALOR_MENSALIDADE']])

In [ ]:
print("\033[92m \n--------------Valores Absolutos--------------------\n", "\033[97m \n", dados.isna().sum())

In [ ]:
#verifica se as classes da variável resposta são balanceadas
COUNT1 = dados['CANCELADO'].value_counts()
COUNT1

In [ ]:
#diferença entre as classes
print("Diferença de: ",COUNT1[0]-COUNT1[1])

In [ ]:
#gráfico countplot
palette=['blue', 'yellow']
ax = sns.countplot(data=dados, x='CANCELADO', hue='CANCELADO', palette=palette)

#legenda
#ax.legend(labels=['não churn', 'churn'])

#rótulos de dados
for p in ax.patches:
    altura = p.get_height()
    ax.text(
        p.get_x() + p.get_width() / 2.,  # posição x centralizada
        altura + 0.5,                    # posição y (acima da barra)
        int(altura),                     # valor da contagem
        ha='center',                     # alinhamento horizontal
        va='bottom'                      # alinhamento vertical
    )
plt.show()

In [ ]:
#quantidade de valores faltantes na variavel alvo por sexo
contagem_por_SEXO = dados.groupby('CANCELADO')['SEXO'].value_counts(dropna=False).reset_index(name='contagem')

print(contagem_por_SEXO)

In [ ]:
print('Percentual de valores faltantes SEXO na classe CANCELADO:', round((1188/59441)*100),"%")
print('Percentual de valores faltantes SEXO na classe NÃO CANCELADO:', round((806/41622)*100),"%")

#**Observação:**

Sendo percentuais baixos iremos remover os valores faltantes para a variável sexo

In [ ]:
#remove os valores faltantes da variável SEXO
dados.dropna(subset=['SEXO'], inplace=True)
dados.isna().sum()

In [ ]:
#quantidade de valores faltantes na variavel alvo
contagem_por_uf = dados.groupby('CANCELADO')['UF'].value_counts(dropna=False).reset_index(name='contagem')
contagem_por_uf.columns = ['CANCELADO', 'UF','Contagem']
display(contagem_por_uf)

In [ ]:
#quantidade de valores para cada classe da variável alvo
dados['CANCELADO'].value_counts()

In [ ]:
print('Percentual de valores faltantes UF na classe CANCELADO:', round((1199/58253)*100),"%")
print('Percentual de valores faltantes UF na classe NÃO CANCELADO:', round((802/40816)*100),"%")

In [ ]:
#remove os valores faltantes da variável UF
dados.dropna(subset=['UF'], inplace=True)
dados.isna().sum()

Tratamento de valores duplicados

In [ ]:
#total de dados duplicados
print(dados.duplicated().sum())
print(f'{dados.duplicated().sum()/dados.shape[0]:.3f}')

In [ ]:
#remove os dados duplicados
dados.drop_duplicates(inplace=True)
dados.shape

In [ ]:
#confere a remoção
print(dados.duplicated().sum())

##Análise gráfica



In [ ]:
#define faixas de tempo de contrato
bins = [0, 250, 311]
labels = ['abaixo de 250 meses', 'acima de 250 meses']

#cria a variável categórica com base na variável contínua
dados['faixa_tempo_plano'] = pd.cut(dados['TEMPO_DE_PLANO_MESES'], bins=bins, labels=labels)
dados.head()

In [ ]:
#tempo de plano pela variável alvo
cont_agrupada_tempo = dados.groupby('CANCELADO')['faixa_tempo_plano'].value_counts()
cont_agrupada_tempo_df = cont_agrupada_tempo.reset_index()
cont_agrupada_tempo_df.columns = ['CANCELADO', 'faixa_tempo_plano','Contagem']
display(cont_agrupada_tempo_df)

In [ ]:
#SEXO pela variável alvo
cont_agrupada_sexo = dados.groupby('CANCELADO')['SEXO'].value_counts()
cont_agrupada_sexo_df = cont_agrupada_sexo.reset_index()
cont_agrupada_sexo_df.columns = ['CANCELADO', 'SEXO','Contagem']
display(cont_agrupada_sexo_df)

In [ ]:
#TITULARRIDADE pela variável alvo
cont_agrupada_TITU = dados.groupby('CANCELADO')['TITULARIDADE'].value_counts()
cont_agrupada_TITU_df = cont_agrupada_TITU.reset_index()
cont_agrupada_TITU_df.columns = ['CANCELADO', 'TITULARIDADE','Contagem']
display(cont_agrupada_TITU_df)

In [ ]:
#inadimplência pela variável alvo
cont_agrupada_INAD = dados.groupby('CANCELADO')['INADIMPLENTE'].value_counts()
cont_agrupada_INAD_df = cont_agrupada_INAD.reset_index()
cont_agrupada_INAD_df.columns = ['CANCELADO', 'INADIMPLENTE','Contagem']
display(cont_agrupada_INAD_df)

In [ ]:
#FAIXA DE RENDA pela variável alvo
cont_agrupada_RENDA = dados.groupby('CANCELADO')['FAIXA_RENDA_USAR'].value_counts()
cont_agrupada_RENDA_df = cont_agrupada_RENDA.reset_index()
cont_agrupada_RENDA_df.columns = ['CANCELADO', 'FAIXA_RENDA_USAR','Contagem']
display(cont_agrupada_RENDA_df)

In [ ]:
#FAIXA DE RENDA pela variável alvo
cont_agrupada_UF = dados.groupby('CANCELADO')['UF'].value_counts()
cont_agrupada_UF_df = cont_agrupada_UF.reset_index()
cont_agrupada_UF_df.columns = ['CANCELADO', 'UF','Contagem']
display(cont_agrupada_UF_df)

In [ ]:
#cria o gráfico de barras
plt.figure(figsize=(8, 5))
sns.barplot(x="faixa_tempo_plano", y="Contagem", hue="CANCELADO", data=cont_agrupada_tempo_df, palette=palette)

#configura os eixos do gráfico
plt.ylabel("Contagem")
plt.xlabel("")
plt.title("Distribuição dos cancelamentos por tempo de plano")
plt.xticks(rotation=0)
plt.legend(title="CANCELADO")

#exibe o gráfico
plt.show()

In [ ]:
#cria o gráfico de barras
plt.figure(figsize=(8, 5))
sns.barplot(x="SEXO", y="Contagem", hue="CANCELADO", data=cont_agrupada_sexo_df, palette=palette)

#configura os eixos do gráfico
plt.ylabel("Contagem")
plt.xlabel("")
plt.title("Distribuição dos cancelamentos por SEXO")
plt.xticks(rotation=0)
plt.legend(title="CANCELADO")

#exibe o gráfico
plt.show()

Teste estatístico para proporção

In [ ]:
# Tabela de frequência: número de cancelamentos por sexo
cancelamentos = dados[dados['CANCELADO'] == 'SIM']['SEXO'].value_counts()

# Total por sexo
total_por_sexo = dados['SEXO'].value_counts()

print("Cancelamentos:\n", cancelamentos)
print("Total por sexo:\n", total_por_sexo)

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

# Valores
count = [cancelamentos['MASCULINO'], cancelamentos['FEMININO']]         # número de sucessos (cancelamentos)
nobs = [total_por_sexo['MASCULINO'], total_por_sexo['FEMININO']]         # número de observações em cada grupo

stat, pval = proportions_ztest(count, nobs)

print(f"Estatística z = {stat:.4f}")
print(f"p-valor = {pval:.4f}")


In [ ]:
#cria o gráfico de barras
plt.figure(figsize=(8, 5))
sns.barplot(x="TITULARIDADE", y="Contagem", hue="CANCELADO", data=cont_agrupada_TITU_df, palette=palette)

#configura os eixos do gráfico
plt.ylabel("Contagem")
plt.xlabel("")
plt.title("Distribuição dos cancelamentos por TITULARIDADE")
plt.xticks(rotation=0)
plt.legend(title="CANCELADO")

#exibe o gráfico
plt.show()

In [ ]:
#cria o gráfico de barras
plt.figure(figsize=(8, 5))
sns.barplot(x="INADIMPLENTE", y="Contagem", hue="CANCELADO", data=cont_agrupada_INAD_df, palette=palette)

#configura os eixos do gráfico
plt.ylabel("Contagem")
plt.xlabel("")
plt.title("Distribuição dos cancelamentos por INADIMPLÊNCIA")
plt.xticks(rotation=0)
plt.legend(title="CANCELADO")

#exibe o gráfico
plt.show()

In [ ]:
#ordem desejada de plotagem
ordem = ['Baixa renda', 'Média renda', 'Alta renda']

#cria o gráfico de barras
plt.figure(figsize=(8, 5))
sns.barplot(x="FAIXA_RENDA_USAR", y="Contagem", hue="CANCELADO", data=cont_agrupada_RENDA_df, palette=palette, order=ordem)

#configura os eixos do gráfico
plt.ylabel("Contagem")
plt.xlabel("")
plt.title("Distribuição dos cancelamentos por FAIXA DE RENDA")
plt.xticks(rotation=0)
plt.legend(title="CANCELADO")

#exibe o gráfico
plt.show()

In [ ]:
#cria o gráfico de barras
plt.figure(figsize=(8, 5))
sns.barplot(y="UF", x="Contagem", hue="CANCELADO", data=cont_agrupada_UF_df, palette=palette)

#configura os eixos do gráfico
plt.ylabel("Contagem")
plt.xlabel("")
plt.title("Distribuição dos cancelamentos por FAIXA DE RENDA")
plt.xticks(rotation=0)
plt.legend(title="CANCELADO")

#exibe o gráfico
plt.show()

Distribuições das variáveis

In [ ]:
# Gráfico de Boxplot
import warnings
warnings.filterwarnings("ignore")

for i in dados.select_dtypes(include="number").columns:
  plt.figure(figsize=(3, 2))  #tamanho da imagem
  sns.boxplot(data=dados, x=i, color="blue")
  plt.show()

Boxplot para verificar outliers

In [ ]:
#histograma com média
for i in dados.select_dtypes(include="number").columns:

    plt.figure(figsize=(3, 2))  #tamanho da imagem

    #histograms
    sns.histplot(data=dados,
                      x=i,
                      bins = 30,
                      hue = 'CANCELADO',
                      palette=palette,
                      alpha=0.8
                )
    #médica em vertical e vermelho
    plt.axvline(np.mean(dados[i]), color="red")

Removendo os outliers

In [ ]:
#selecionando os nomes da variáveis numéricas
dados.select_dtypes(include="number").columns

In [ ]:
#função para calcular os percentis das variáveis e o interquartil
def out (col):
  q1, q3 = np.percentile(col, [25, 75])
  iqr=q3-q1
  lw=q1-1.5*iqr
  uw=q3+1.5*iqr
  return lw, uw

In [ ]:
#remove os outliers
for i in ['IDADE_NA_ADESAO', 'TEMPO_DE_PLANO_MESES','QTD_CONSULTAS_12M', 'QTD_INTERNACOES_12M', 'VALOR_MENSALIDADE']:
  lw,uw=out(dados[i])
  dados[i]=np.where(dados[i]<lw, lw, dados[i])
  dados[i]=np.where(dados[i]>uw, uw, dados[i])

Verificando se os outliers foram removidos

In [ ]:
#grafico boxplot

for i in ['IDADE_NA_ADESAO', 'TEMPO_DE_PLANO_MESES','QTD_CONSULTAS_12M', 'QTD_INTERNACOES_12M', 'VALOR_MENSALIDADE']:
  plt.figure(figsize=(3, 2))  #tamanho da imagem
  sns.boxplot(data=dados, x=i, color="blue")
  plt.show()

In [ ]:
#verificando o tamanho final do banco de dados após a limpeza
dados.shape

In [ ]:
#verifica se as classes da variável resposta são balanceadas
COUNT2 = dados['CANCELADO'].value_counts()
COUNT2

In [ ]:
#gráfico de barras Churn
ax = sns.countplot(data=dados, x='CANCELADO', hue='CANCELADO', palette=palette)

#legenda
#ax.legend(labels=['não churn', 'churn'])

#rótulos de dados
for p in ax.patches:
    altura = p.get_height()
    ax.text(
        p.get_x() + p.get_width() / 2.,  # posição x centralizada
        altura + 0.5,                    # posição y (acima da barra)
        int(altura),                     # valor da contagem
        ha='center',                     # alinhamento horizontal
        va='bottom'                      # alinhamento vertical
    )
plt.show()

In [ ]:
print("Diferença de: ",COUNT1[0]-COUNT2[0], "na classe CHURN")
print("Diferença de: ",COUNT1[1]-COUNT2[1], 'na classe NÃO CHURN')

Gráfico de dispersão para entender a relação das variáveis explicativas com a variável resposta

In [ ]:
# Gráfico de Dispersão
import warnings
warnings.filterwarnings("ignore")

for i in dados.select_dtypes(include="number").columns:
  plt.figure(figsize=(3, 2))  #tamanho da imagem
  sns.scatterplot(data=dados, x=i, y= "CANCELADO", color="blue")
  plt.show()

*Observação: clientes com mais de 250 meses de contrato tem maior chance de não cancelar o plano*

In [ ]:
dados.head()

In [ ]:
#remove colunas indesejadas
dados_modelo = dados.drop(columns=['FAIXA_RENDA_USAR','faixa_tempo_plano','CANCELADO'])
#cria dummies
dados_modelo = pd.get_dummies(data=dados_modelo, columns=['TITULARIDADE', 'FAIXA_RENDA', 'SEXO', 'UF',
       'INADIMPLENTE'], drop_first= True)
dados_modelo


Separação da base em treino e teste

In [ ]:
#separação em X e y
X = dados_modelo.drop(columns=['ID_CLIENTE', 'CANCELADO_USAR'], axis=1).copy()
y = dados_modelo['CANCELADO_USAR'].copy()

In [ ]:
#separa os dados em conjuntos de treinamento e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [ ]:
y_train.value_counts()

Padronização dos dados

In [ ]:
#padroniza as colunas
padronizador = StandardScaler()

In [ ]:
#Separa as variáveis numéricas e categóricas
num_cols = X_train.select_dtypes(include=['number']).columns
cat_cols = X_train.select_dtypes(exclude=['number']).columns

In [ ]:
#inicia o scaler
scaler = StandardScaler()

#ajusta apenas no treino para evitar o vazamento de dados (data leakage)
X_train_num_padronizado = pd.DataFrame(
    #transforma o treino
    scaler.fit_transform(X_train[num_cols]),
    columns=num_cols,
    index=X_train.index
)

In [ ]:
#transforma o teste
X_test_num_padronizado = pd.DataFrame(
    scaler.transform(X_test[num_cols]),
    columns=num_cols,
    index=X_test.index
)

In [ ]:
#separa as variáveis categóricas originais
X_train_cat = X_train[cat_cols].copy()
X_test_cat = X_test[cat_cols].copy()

#concatena os dados novamente
X_train_padronizado = pd.concat([X_train_num_padronizado, X_train_cat], axis=1)
X_test_padronizado = pd.concat([X_test_num_padronizado, X_test_cat], axis=1)


In [ ]:
X_train_padronizado.head()

##Random Forest

Etapa de ajuste e validação do modelo

In [ ]:
#dicionário de parâmetros para ser utilizado no gridsearch
params = {'max_depth':[2,3,4,5],
          'class_weight':[None, 'balance']}

#armazenando o modelo na variável model
Randon_Forest = RandomForestClassifier()
model_RF = GridSearchCV(estimator = Randon_Forest, param_grid = params, cv= 5, scoring= 'roc_auc')

#ajustando o modelo com o gridSearch (poderia ser o optuna)
model_RF.fit(X_train_padronizado, y_train)

In [ ]:
#testando o modelo
y_pred_proba_RF = model_RF.predict_proba(X_test_padronizado)
y_pred_RF = model_RF.predict(X_test_padronizado)
#área abaixo da curva roc calculada para base de teste
auc = roc_auc_score(y_test, y_pred_proba_RF[:, 1])

In [ ]:
#parâmetros e métricas do modelo de ajuste e teste
print(f'Parâmetros : {model_RF.best_params_}')
print(f'AUC Treino : {model_RF.best_score_:.2f}')
print(f'AUC Teste : {auc:.2f}')

Teste do modelo final

In [ ]:
# utiliza melhores parâmetros testados anteriormente
params = {'max_depth':[5],
          'class_weight':[None]}
#modelo
Random_Forest = RandomForestClassifier()
model_RF = GridSearchCV(estimator = Random_Forest, param_grid = params, cv= 10, scoring= 'accuracy')

#ajustando o modelo
model_RF.fit(X_train_padronizado, y_train)



In [ ]:
# dataframe de previsao para a matriz de confusão
matriz_confusao_RF = pd.DataFrame({'observado': y_test, 'predito': y_pred_proba_RF[:, 1]})
matriz_confusao_RF

In [ ]:
#ponto de corte ajustável para avaliação da matriz de confusão e os respectivos erros tipo I e tipo II
#construção de função para a definição da matriz de confusão

def matriz_confusao(predicts, observado, cutoff):
    values = predicts.values
    predicao_binaria = []
    for item in values:
        if item < cutoff:
            predicao_binaria.append(0)
        else:
            predicao_binaria.append(1)
    cm = confusion_matrix(predicao_binaria, observado)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.xlabel('Verdadeiro')
    plt.ylabel('Classificado')
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()
    plt.show()
    sensitividade = recall_score(observado, predicao_binaria, pos_label=1)
    especificidade = recall_score(observado, predicao_binaria, pos_label=0)
    acuracia = accuracy_score(observado, predicao_binaria)
    # Visualização dos principais indicadores desta matriz de confusão
    indicadores = pd.DataFrame({'Sensitividade':[sensitividade],
                                'Especificidade':[especificidade],
                                'Acurácia':[acuracia]})
    return indicadores

In [ ]:
#matriz de confusão
matriz_confusao(observado= matriz_confusao_RF['observado'],
                predicts= matriz_confusao_RF['predito'],
                cutoff=0.5)

In [ ]:
print(f'Acurácia de treino: {model_RF.best_score_:.2f}')
#acurácia de teste
acuracia_teste = accuracy_score(y_test, y_pred_RF)

#Avaliar o modelo
print(f"\033[92m Acurácia de teste: {acuracia_teste:.2f}")
print("\033[91m \n----------------------------------\n")
print("\033[92m Relatório de Classificação:\n", classification_report(y_test, y_pred_RF))
print("\033[91m \n----------------------------------\n")
print("\033[92m Precisão:\n", round(precision_score(y_test, y_pred_RF),2))


In [ ]:
#curva ROC e a área abaixo da curva
fpr, tpr, thresholds = roc_curve(y_test, y_pred_RF)
roc_auc = roc_auc_score(y_test, y_pred_RF)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='yellow', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de falso positivo')
plt.ylabel('Taxa de verdadeiro positivo')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:
# gráfico de barras comparando a acurácia de treino e teste
categories = ['Treino', 'Teste']
values = [99, 95]

fig, ax = plt.subplots()
bar_container = ax.bar(categories, values, color='blue')
ax.set(ylabel='', title='Acurácia Treino e Teste - Random Forest', ylim=(0, 120))
ax.bar_label(bar_container, fmt='{:,.0f}%')
plt.show()

In [ ]:
# armazena 0 melhor modelo
melhor_modelo_RF = model_RF.best_estimator_

#grau de importância das variáveis
importances = melhor_modelo_RF.feature_importances_

#dataFrame com nomes das variáveis e suas importâncias
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Exibe as variáveis ordenadas por grau de importância

display(feature_importance)

##Regressão logística

In [ ]:
#modelo de Regressão Logística
logistic_model = LogisticRegression(max_iter=1000)

# Definir os parâmetros a serem testados

param_grid_RL = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['lbfgs', 'saga', 'liblinear']
}

strat_RL = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Criar o objeto GridSearchCV

model_RL = GridSearchCV(logistic_model, param_grid_RL, cv=strat_RL, scoring='accuracy', verbose=1, n_jobs=-1)

# Treinar o modelo com a busca de hiperparâmetros
model_RL.fit(X_train_padronizado, y_train)

#print("Melhores hiperparâmetros:", model.best_params_)

In [ ]:
print("\033[92m Melhores hiperparâmetros:\n\n", "\033[97m", model_RL.best_params_)
coefficients = model_RL.best_estimator_.coef_
coefficients

In [ ]:
#nomes das colunas
nomes_variaveis = X.columns
nomes_variaveis

In [ ]:
#juntando as variáveis com seus respectivos coeficientes
feature_coefficients = dict(zip(nomes_variaveis, coefficients[0]))

#dataframe dos coeficientes e variáveis
coefficients_df = pd.DataFrame({'Variável': nomes_variaveis, 'Coeficiente': coefficients[0]}).sort_values(by='Coeficiente', ascending=False)
coefficients_df

In [ ]:
#previsões
y_pred_proba_RL = model_RL.predict_proba(X_test_padronizado)[:, 1]       #pegando apenas a segunda coluna (probabilidade de evento)
y_pred_RL = model_RL.predict(X_test_padronizado)

In [ ]:
# dataframe de previsao para a matriz de confusão
confusao_df = pd.DataFrame({'observado': y_test, 'predito': y_pred_proba_RL})
confusao_df

In [ ]:
#matrize de confusão
matriz_confusao(observado= confusao_df['observado'],
                predicts= confusao_df['predito'],
                cutoff=0.7)

In [ ]:
#Avaliar o modelo

acuracia_teste_RL = accuracy_score(y_test, y_pred_RL)

print(f'Acurácia de treino: {model_RL.score(X_train_padronizado, y_train):.2f}')
print(f"\033[92mAcurácia de teste: {acuracia_teste_RL:.2f}")
print(f"\033[91m\n----------------------------------\n")
print(f"\033[92mRelatório de Classificação:\n{classification_report(y_test, y_pred_RL)}")
print(f"\033[92mPrecisão: {precision_score(y_test, y_pred_RL):.2f}")
print(f"\033[92mRecall: {recall_score(y_test, y_pred_RL):.2f}")


In [ ]:
#Curva ROC e AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_RL)
auc_score_RL = roc_auc_score(y_test, y_pred_RL)
print("AUC:", auc_score_RL)

plt.plot(fpr, tpr, color='yellow', label=f'Curva ROC (AUC = {auc_score_RL:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel("Taxa de Falsos Positivos")
plt.ylabel("Taxa de Verdadeiros Positivos")
plt.title("Curva ROC")
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:
# gráfico de barras comparando a acurácia de treino e teste
categories = ['Treino', 'Teste']
values = [95, 95]

fig, ax = plt.subplots()
bar_container = ax.bar(categories, values, color='blue')
ax.set(ylabel='', title='Acurácia Treino e Teste - Random Forest', ylim=(0, 120))
ax.bar_label(bar_container, fmt='{:,.0f}%')
plt.show()